# Solving Hanoi Tower with AI

The read of the illusion of thinking (from Apple's research) found reasonance in my own fun in problem solving. At a certain problem complexity, I give up when not equipped with the right tools. Perhaps then my problem solving is not much about thinking about the problem, but thinking about the tools to solve the problem. So it lead me to this question:

- Does non thinking AI solves puzzle better than thinking one when provided with the right tools?
- Does thinking AI are able to pick up the right tools to solve it?

In this notebook, I wanted to explore the first question based on reading about mcp (link to medium), hanoi algorithm (link to medium), and the Illusion of thinking article (link to article)


In this experiment, we'll:
1. Set up an MCP (Model Context Protocol) server for puzzle validation
2. Configure an AI agent to solve the Tower of Hanoi puzzle
3. Compare different approaches (with/without MCP, with/without pseudocode)
4. Look at the success rate of the AI agent

The Tower of Hanoi serves as an excellent test case because:
- It has a clear, well-defined solution
- It requires systematic thinking and planning
- It can be validated step-by-step

## Hanoi MCP server

The server provides a hanoi tower puzzle solver, a python version of the following pseudo code algorithm

```
ALGORITHM Solve(n, source, target, auxiliary, moves)
    // n = number of disks to move
    // source = starting peg (0, 1, or 2)
    // target = destination peg (0, 1, or 2)
    // auxiliary = the unused peg (0, 1, or 2)
    // moves = list to store the sequence of moves

    IF n equals 1 THEN
        // Get the top disk from source peg
        disk = the top disk on the source peg
        // Add the move to our list: [disk_id, source, target]
        ADD [disk, source, target] to moves
        RETURN
    END IF

    // Move n-1 disks from source to auxiliary peg
    Solve(n-1, source, auxiliary, target, moves)

    // Move the nth disk from source to target
    disk = the top disk on the source peg
    ADD [disk, source, target] to moves

    // Move n-1 disks from auxiliary to target
    Solve(n-1, auxiliary, target, source, moves)

    END ALGORITHM
```

In [1]:
import multiprocessing
from server.hanoi import run_mcp_server

multiprocessing.Process(target=run_mcp_server).start() 

## Example 

In [2]:
from config.hanoi_config import HanoiConfig, HanoiSolution, HanoiMove
from client.client_hanoi_tower import run_agent
from itertools import product
import pandas as pd
from datetime import datetime
import os
from tqdm import tqdm
import pickle

config = HanoiConfig(n_disks = 2)
config.use_mcp = True
config.add_pseudocode = True
config.mcp_version = 1
config.model_name = 'o3-mini'# 'gpt-4o-mini'
config.server_command = "python"
config.server_args = ["server/hanoi.py"]

[06/21/25 14:42:35] INFO     Starting Hanoi MCP server               ]8;id=403358;file:///Users/olivierbertrand/my-test-project/ai-hanoi-mcp/server/hanoi.py\hanoi.py]8;;\:]8;id=888983;file:///Users/olivierbertrand/my-test-project/ai-hanoi-mcp/server/hanoi.py#69\69]8;;\


In [3]:
# Use await instead of asyncio.run() in Jupyter notebooks
result = await run_agent(config=config)

[06/21/25 14:42:38] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=29190;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=137313;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

[06/21/25 14:42:44] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=6106;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=683732;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

[06/21/25 14:42:48] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=20861;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=478206;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

In [4]:
if "structured_response" in result:
        print("\nStructured solution:")
        solution = result["structured_response"]
        print(f"Total moves: {solution.total_moves}")
        valid_solution = solution.validate_solution(config.n_disks)
        if not valid_solution['is_valid']:
                print(f"Invalid solution: {valid_solution}")
        else:
                print("Valid solution")
else:
        print('No structured solution')


Structured solution:
Total moves: 3
Valid solution


In [5]:
result


{'messages': [HumanMessage(content='\n    I have a puzzle with 2 disks of different sizes with\n    Initial configuration:\n    • Peg 0: 2 (bottom), ... 2, 1 (top)\n    • Peg 1: (empty)\n    • Peg 2: (empty)\n    Goal configuration:\n    • Peg 0: (empty)\n    • Peg 1: (empty)\n    • Peg 2: 2 (bottom), ... 2, 1 (top)\n    Rules:\n    • Only one disk can be moved at a time.\n    • Only the top disk from any stack can be moved.\n    • A larger disk may not be placed on top of a smaller disk.\n    Find the sequence of moves to transform the initial configuration into the goal configuration.\n    ', additional_kwargs={}, response_metadata={}, id='77cd91be-6580-4e09-9658-d0ce8593f9c1'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kFty2Ofmzqsp2aLAWFJGo9lF', 'function': {'arguments': '{"n": 2}', 'name': 'hanoi_solver'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 1007, 'total_tokens': 1101, 'com

## Experiment

In [6]:
mcp_version = 1
saving_result_file = f'data/hanoi_results_v{mcp_version}.csv'
if os.path.exists(saving_result_file):
    completed_results = pd.read_csv(saving_result_file)
    completed_results = completed_results.loc[
        :, ['model_name', 'use_mcp', 'add_pseudocode', 'n_disks', 'ith_try']
    ]
else:
    completed_results = None


In [7]:
if mcp_version == 1:
    disks = [10, 8, 6, 4, 3, 2]
else:
    disks = [10, 8, 6]
tries = range(15)   
models = ['o4-mini', 'gpt-4.1-mini']
if mcp_version == 1:
    helpers = [
        dict(use_mcp = False, add_pseudocode = False),
        dict(use_mcp = False, add_pseudocode = True),
        dict(use_mcp = True, add_pseudocode = False)
    ]
else:
    helpers = [
        dict(use_mcp = True, add_pseudocode = False)
    ]

In [8]:
configs = []
for n_disk, model, ith_try, helper in product(disks, models, tries, helpers):
    configs.append(dict(n_disks = n_disk, model_name = model, ith_try = ith_try, use_mcp = helper['use_mcp'], add_pseudocode = helper['add_pseudocode']))
configs = pd.DataFrame(configs)
configs.set_index(['n_disks', 'model_name', 'ith_try', 'use_mcp', 'add_pseudocode'], inplace=True)
completed_results.set_index(['n_disks', 'model_name', 'ith_try', 'use_mcp', 'add_pseudocode'], inplace=True)
configs = configs.loc[~configs.index.isin(completed_results.index)]
configs.reset_index(inplace=True)

In [9]:



for _, config_series in tqdm(configs.iterrows(), total=len(configs)):
    config = HanoiConfig(n_disks = config_series.n_disks)
    config.use_mcp = config_series.use_mcp
    config.add_pseudocode = config_series.add_pseudocode
    config.model_name = config_series.model_name
    config.mcp_version = mcp_version
    config.server_command = "python"
    config.server_args = ["server/hanoi.py"]

    run_start = datetime.now()
    result = await run_agent(config)
    run_end = datetime.now()
    # To identify with the meta parameters
    result['run_start'] = run_start
    result['run_end'] = run_end
    with open('data/hanoi_results.pkl', 'ab') as f:
        pickle.dump(result, f)
    has_structured_response = "structured_response" in result
    if has_structured_response:
        solution = result["structured_response"]
        valid_solution = solution.validate_solution(config.n_disks)['is_valid']
    else:
        valid_solution = False

    to_save = pd.DataFrame(
        [dict(
            model_name = config.model_name,
            use_mcp = config.use_mcp,
            add_pseudocode = config.add_pseudocode,
            n_disks = config.n_disks,
            ith_try = config_series.ith_try,
            has_structured_response = has_structured_response,
            valid_solution = valid_solution,
            total_moves = solution.total_moves if has_structured_response else None,
            run_start = run_start,
            run_end = run_end
        )]
    )
    to_save.to_csv(saving_result_file, mode='a', header=not os.path.exists(saving_result_file))

  0%|          | 0/29 [00:00<?, ?it/s]

[06/21/25 14:43:04] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=874710;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=768440;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

[06/21/25 14:43:10] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=790925;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=66636;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

  3%|▎         | 1/29 [00:21<10:06, 21.66s/it]

[06/21/25 14:43:22] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=690661;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=541945;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

...

 38%|███▊      | 11/29 [03:57<06:01, 20.08s/it]

[06/21/25 14:47:11] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=546852;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=417224;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

[06/21/25 14:47:18] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=708257;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=672742;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

 41%|████▏     | 12/29 [04:47<06:47, 23.99s/it]


CancelledError: 

# Results

In [ ]:
results = pd.read_csv(saving_result_file)
results.set_index(['n_disks', 'model_name', 'use_mcp', 'add_pseudocode', 'ith_try'], inplace=True)
results.loc[4,'gpt-4.1-mini',False,False]

/var/folders/b3/p5r945115lb4b0k1m4r15f780000gn/T/ipykernel_10864/3073076946.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  results.loc[4,'gpt-4.1-mini',False,False]


,Unnamed: 0,has_structured_response,valid_solution,total_moves,run_start,run_end
ith_try,,,,,,
14,0,True,True,15,2025-06-21 14:37:46.865523,2025-06-21 14:37:59.885946
14,0,True,False,15,2025-06-21 14:37:59.896433,2025-06-21 14:38:18.467719
14,0,True,False,15,2025-06-21 14:38:18.473545,2025-06-21 14:38:30.180917
14,0,True,False,15,2025-06-21 14:38:30.185532,2025-06-21 14:38:42.488763
14,0,True,False,15,2025-06-21 14:38:42.493895,2025-06-21 14:38:55.086835
14,0,True,True,15,2025-06-21 14:38:55.091586,2025-06-21 14:39:11.555288
14,0,True,False,15,2025-06-21 14:39:11.559951,2025-06-21 14:39:20.863491
14,0,True,True,15,2025-06-21 14:39:20.870,2025-06-21 14:39:38.498889
14,0,True,False,15,2025-06-21 14:39:38.503056,2025-06-21 14:39:50.061195


In [ ]:
# Calculate success rate for each configuration
success_rates = results.groupby(['model_name', 'use_mcp', 'add_pseudocode', 'n_disks']).agg({
    'valid_solution': ['count', 'sum', lambda x: (x.sum() / x.count() * 100).round(2)]
}).round(2)

# Rename columns for clarity
success_rates.columns = ['total_attempts', 'successful_attempts', 'success_rate_percent']
success_rates = success_rates.reset_index()
success_rates.head()

,model_name,use_mcp,add_pseudocode,n_disks,total_attempts,successful_attempts,success_rate_percent
0,gpt-4.1-mini,False,False,2,15,15,100.00
1,gpt-4.1-mini,False,False,3,15,15,100.00
2,gpt-4.1-mini,False,False,4,15,5,33.33
3,gpt-4.1-mini,False,False,5,15,13,86.67
4,gpt-4.1-mini,False,False,6,15,1,6.67


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots for the three different configurations
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Tool ✗, Pseudocode ✗', 'Tool ✗, Pseudocode ✓', 'Tool ✓, Pseudocode ✗'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}]]
)

# Define the three configurations
configs = [
    {'use_mcp': False, 'add_pseudocode': False},
    {'use_mcp': False, 'add_pseudocode': True},
    {'use_mcp': True, 'add_pseudocode': False}
]

# Colors for the models
colors = {'o4-mini': 'blue', 'gpt-4.1-mini': 'red'}

for col, config in enumerate(configs, 1):
    # Filter data for this configuration
    mask = (success_rates['use_mcp'] == config['use_mcp']) & \
           (success_rates['add_pseudocode'] == config['add_pseudocode'])
    config_data = success_rates[mask]
    
    # Plot each model
    for model in ['o4-mini', 'gpt-4.1-mini']:
        model_data = config_data[config_data['model_name'] == model]
        
        fig.add_trace(
            go.Scatter(
                x=model_data['n_disks'],
                y=model_data['success_rate_percent'],
                mode='lines+markers',
                name=f'{model}',
                line=dict(color=colors[model]),
                showlegend=(col == 1),  # Only show legend for first subplot
                hovertemplate='<b>%{fullData.name}</b><br>' +
                            'Disks: %{x}<br>' +
                            'Success Rate: %{y:.1f}%<br>' +
                            '<extra></extra>'
            ),
            row=1, col=col
        )

# Update layout
fig.update_layout(
    title='Hanoi Tower Success Rates by Configuration',
    height=500,
    width=1200,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Update axes labels
for i in range(1, 4):
    fig.update_xaxes(title_text="Number of Disks", row=1, col=i)
    fig.update_yaxes(title_text="Success Rate (%)", row=1, col=i)

fig.show()
